# Getting Started with ctapipe

This notebook is a mixture of a hands-on that was originally presented at the Paris CTA Consoritum meeting by K. Kosack (@kosack) and another one presented by M. Noethe (@maxnoe)

Modified version by R. Lopez-Coto (@rlopezcoto) (16/02/21)

## 1. Load and loop over data

In [ ]:
from ctapipe.io import event_source
from ctapipe import utils
from matplotlib import pyplot as plt
import numpy as np
%matplotlib inline

Config to get nicer plots for demonstrations

In [ ]:
plt.rcParams['figure.figsize'] = (10, 8)
plt.rcParams['font.size'] = 20
plt.rcParams['figure.figsize']

In [ ]:
path = utils.get_dataset_path("gamma_test_large.simtel.gz")
source = event_source(path, max_events=4)

In [ ]:
for event in source:
    print(event.count, event.index.event_id, event.mc.energy)

In [ ]:
event

In [ ]:
event.r0

In [ ]:
for event in event_source(path, max_events=4):
    print(event.count, event.r0.tels_with_data)

In [ ]:
event.r0.tel[2]

In [ ]:
r0tel = event.r0.tel[2]

In [ ]:
r0tel.waveform

In [ ]:
r0tel.waveform.shape

note that this is ($N_{channels}$, $N_{pixels}$, $N_{samples}$)

In [ ]:
plt.pcolormesh(r0tel.waveform[0])

In [ ]:
plt.plot(r0tel.waveform[0,10])

In [ ]:
len(r0tel.waveform[0,10])

In [ ]:
from ipywidgets import interact

@interact
def view_waveform(chan=0, pix_id=200):
    plt.plot(r0tel.waveform[chan, pix_id])

try making this compare 2 waveforms

## 2. Explore the instrument description
This is all well and good, but we don't really know what camera or telescope this is... how do we get instrumental description info?

Currently this is returned *inside* the event (it will soon change to be separate in next version or so)

In [ ]:
subarray = source.subarray

In [ ]:
subarray.info()

In [ ]:
subarray.peek()

In [ ]:
subarray.to_table()

In [ ]:
subarray.tel[2]

In [ ]:
subarray.tel[2].camera

In [ ]:
subarray.tel[2].optics

In [ ]:
tel = subarray.tel[2]

In [ ]:
tel.camera

In [ ]:
tel.optics

In [ ]:
tel.camera.geometry.pix_x

In [ ]:
tel.camera.geometry.to_table()

In [ ]:
tel.optics.mirror_area

In [ ]:
from ctapipe.visualization import CameraDisplay

In [ ]:
disp = CameraDisplay(tel.camera.geometry)

In [ ]:
disp = CameraDisplay(tel.camera.geometry)
disp.image = r0tel.waveform[0,:,10]  # display gain 0, sample 0 (try others like 10)
disp.add_colorbar()

## 3. Apply some calibration and trace integration

In [ ]:
from ctapipe.calib import CameraCalibrator

In [ ]:
calib = CameraCalibrator(subarray)

In [ ]:
for event in event_source(path, max_events=4):
    calib(event) # fills in r1, dl0, and dl1
    print(event.dl1.tel.keys())

In [ ]:
event.dl1.tel[2]

In [ ]:
dl1tel = event.dl1.tel[2]

In [ ]:
dl1tel.image.shape # note this will be gain-selected in next version, so will be just 1D array of 1855

In [ ]:
dl1tel.peak_time

In [ ]:
CameraDisplay(tel.camera.geometry, image=dl1tel.image)

In [ ]:
CameraDisplay(tel.camera.geometry, image=dl1tel.peak_time)

In [ ]:
disp = CameraDisplay(cam_geom)
disp.image = image
disp.highlight_pixels(cam_geom.get_border_pixel_mask(1), linewidth=2, color='xkcd:red')

# 4. Image Cleaning and Parameters

In [ ]:
from ctapipe.image import tailcuts_clean

In [ ]:
cleaning_level = {
    'ASTRICam': (5, 7, 2),  # (5, 10)?
    'LSTCam': (3, 6, 2),  # ?? (3, 6) for Abelardo...
    'FlashCam': (4, 8, 2),  # there is some scaling missing?
}

In [ ]:
image = dl1tel.image
peak_time = dl1tel.peak_time
cam_geom = tel.camera.geometry

boundary, picture, min_neighbors = cleaning_level[subarray.camera_types[0].camera_name]

clean_mask = tailcuts_clean(
    cam_geom, 
    image,
    boundary_thresh=boundary,
    picture_thresh=picture,
    min_number_picture_neighbors=min_neighbors
)
clean_mask

In [ ]:
CameraDisplay(cam_geom, image=clean_mask)

In [ ]:
image_cleaned = image.copy()
image_cleaned[~clean_mask] = 0.0

peak_time_cleaned = peak_time.copy()
peak_time_cleaned[~clean_mask] = 0.0

In [ ]:
disp = CameraDisplay(tel.camera.geometry, image=peak_time_cleaned)
disp.add_colorbar()

In [ ]:
disp = CameraDisplay(cam_geom, image=image_cleaned)
disp.cmap = plt.cm.coolwarm
disp.add_colorbar()
plt.xlim(-1.0,0)
plt.ylim(0,1.0)

### Image parameters

In [ ]:
hillas_params = hillas_parameters(cam_geom, image_cleaned)
print(hillas_params)

In [ ]:
disp = CameraDisplay(cam_geom, image=image_cleaned)
disp.cmap = plt.cm.coolwarm
disp.add_colorbar()
plt.xlim(-1.0,0)
plt.ylim(0,1.0)
disp.overlay_moments(params, color='white', lw=2)

In [ ]:
from ctapipe.image import hillas_parameters, leakage, concentration
from ctapipe.image.timing import timing_parameters
from ctapipe.image.morphology import number_of_islands
from ctapipe.image.hillas import camera_to_shower_coordinates

In [ ]:
timing = timing_parameters(
    cam_geom,
    image_cleaned,
    peak_time_cleaned,
    hillas_params,
    clean_mask
)

print(timing)

In [ ]:
long, trans = camera_to_shower_coordinates(
    cam_geom.pix_x, cam_geom.pix_y,
    hillas_params.x, hillas_params.y, hillas_params.psi
)

plt.plot(long[clean_mask], peak_time[clean_mask], 'o')
plt.plot(long[clean_mask], timing.slope * long[clean_mask] + timing.intercept)

In [ ]:
l = leakage(cam_geom, image, clean_mask)
print(l)

In [ ]:
disp = CameraDisplay(cam_geom)
disp.image = image
disp.highlight_pixels(clean_mask, linewidth=2, color='xkcd:red')

In [ ]:
n_islands, island_id = number_of_islands(cam_geom, clean_mask)

print(n_islands)

In [ ]:
conc = concentration(cam_geom, image, hillas_params)
print(conc)

## 5.  Let's put it all together: 
- loop over events, selecting only telescopes of the same type (e.g. LST:LSTCam)
- for each event, apply calibration/trace integration
- calculate Hillas parameters 
- write out all hillas paremeters to a file that can be loaded with Pandas

first let's select only those telescopes with LST:LSTCam

In [ ]:
subarray.telescope_types

In [ ]:
subarray.get_tel_ids_for_type("LST_LST_LSTCam")

Now let's write out a program

In [ ]:
data = utils.get_dataset_path("gamma_test_large.simtel.gz") 
source = event_source(data, allowed_tels=[1,2,3,4],  max_events=10) # remove the max_events limit to get more stats
for event in source:
    calib(event)
    
    for tel_id, tel_data in event.dl1.tel.items():
        tel = subarray.tel[tel_id]
        mask = tailcuts_clean(tel.camera.geometry, tel_data.image)
        params = hillas_parameters(tel.camera.geometry[mask], tel_data.image[mask])

In [ ]:
from ctapipe.io import HDF5TableWriter


In [ ]:
with HDF5TableWriter(filename='hillas.h5', group_name='dl1', overwrite=True) as writer:
    
    for event in event_source(data, allowed_tels=[1,2,3,4],  max_events=10):
        calib(event)
    
        for tel_id, tel_data in event.dl1.tel.items():
            tel = subarray.tel[tel_id]
            mask = tailcuts_clean(tel.camera.geometry, tel_data.image)
            params = hillas_parameters(tel.camera.geometry[mask], tel_data.image[mask])
            writer.write("hillas", params)

### We can now load in the file we created and plot it

In [ ]:
!ls *.h5

In [ ]:
import pandas as pd

hillas = pd.read_hdf("hillas.h5", key='/dl1/hillas')
hillas

If you do this yourself, loop over more events to get better statistics

# Exercises

1. Plot together several waveforms of an LST
2. Display in a camera the DL1 information of all LST events in a file
3. Plot a histogram of all hillas parameters of a telescope type